In [ ]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import extract_info
import parameters as params
import download_and_process_pdf as dp

In [ ]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [ ]:
# extract text from given .pdf file
def pdf2text(pdf_path, text_path, page_start, page_end):
    try:
        # creating a pdf reader object
        reader = PyPDF2.PdfReader(pdf_path)
        
        # printing number of pages in pdf file
        page_max = len(reader.pages)
        
        # getting a specific page from the pdf file
        text = ""
        
        for i in range(page_start, page_end + 1):
            # print(i)
            page = reader.pages[i]
            text = text + "".join(page.extract_text().splitlines())

        with open(text_path, "w") as f:
            f.write(text)
        f.close()

        return True
    except:
        return False
# --------------------start of test code--------------------
# ind = 1
# pdf_file_name = str(index) + ".pdf"
# pdf_folder = fpath.litera_pdf_folder
# pdf_path = os.path.join(pdf_folder, pdf_file_name)

# text_folder_path = fpath.litera_text_folder
# text_file_name = pdf_file_name.split(".pdf")[0] + ".txt"
# text_path = os.path.join(text_folder_path, text_file_name)

# start_page = 0
# end_page = 1
# if not pdf2text(pdf_path, text_path, start_page, end_page):
#     print("Error: pdf2text() failed")
# ---------------------end of test code---------------------

In [ ]:
pdf_url = "https://journals.physiology.org/doi/10.1152/jn.2001.85.1.219"
# pdf_url = "https://jnm.snmjournals.org/content/jnumed/39/2/281.full.pdf"
# doi = "10.1016/0006-8993(95)01338-5"
# pdf_url = "https://physoc.onlinelibrary.wiley.com/doi/epdf/10.1113/JP280844"
# pdf_url = "https://www.microbiologyresearch.org/content/journal/jgv/10.1099/vir.0.79883-0"
# pdf_url = "https://europepmc.org/article/med/8784824"
# pdf_url = "https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3689615"
# pdf_url = "https://www.ingentaconnect.com/content/aalas/cm/2000/00000050/00000002/art00006"
# pdf_url = "https://journals.lww.com/neuroreport/abstract/1994/10000/further_evidence_for_two_types_of_corticopulvinar.6.aspx"
# # pdf_url = "https://www.ahajournals.org/doi/reader/10.1161/01.STR.0000087786.38997.9E"
# # pdf_url = "https://iovs.arvojournals.org//arvo/content_public/journal/iovs/934840/i1552-5783-57-1-1.pdf"
# pdf_url = "https://royalsocietypublishing.org/doi/10.1098/rspb.1953.0054"
# "https://jamanetwork.com/journals/jamaneurology/article-abstract/565945"
ind = 10
pdf_folder = fpath.pdf_folder
dp.download_and_rename_pdf(pdf_url, ind, pdf_folder)